# Hyperparameter Optimization 과제

by. DA팀 16기 김지환

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## <심리 성향 기반 투표 여부 예측>
https://dacon.io/competitions/official/235647/overview/

#### - 심리 테스트 결과를 기반으로 해당 인원이 투표를 했는 지 예측하는 task 입니다.
#### - 변수에 대한 설명은 첨부한 텍스트파일을 참고해주세요.
#### - Cross validation 했을 때 점수가 0.69 이상 나오면 성공입니다.

In [2]:
!pip install catboost

In [3]:
!pip install bayesian-optimization

In [4]:
!pip install git+https://github.com/thuijskens/scikit-hyperband.git

  Cloning https://github.com/thuijskens/scikit-hyperband.git to /tmp/pip-req-build-389nwjpo
  Running command git clone -q https://github.com/thuijskens/scikit-hyperband.git /tmp/pip-req-build-389nwjpo
  Created wheel for scikit-hyperband: filename=scikit_hyperband-0.0.1-cp36-none-any.whl size=10362 sha256=4dac164d18bbb71003774e950c193226fc798dae76a13ab451d014015d7cf1db
  Stored in directory: /tmp/pip-ephem-wheel-cache-d90wjdrz/wheels/9f/b0/a4/090ef56209908e0e7d583e47d3935f14593d31d9491b93cd3e
Successfully built scikit-hyperband


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from tqdm import tqdm
from sklearn.model_selection import KFold
from functools import partial
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import OneHotEncoder
import warnings        
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#### - 간단한 전처리를 진행합니다.

In [6]:
df = pd.read_csv('/content/drive/My Drive/YBIGTA/DA 교육세션/2020-11-07 FE   Parameter tuning/train.csv', index_col=0)

In [7]:
df = df.applymap(lambda x: x.replace('s', '') if type(x)==str else x)

In [8]:
# One-hot encoding
enc = OneHotEncoder()
df['gender'] = enc.fit_transform(df[['gender']]).toarray()
df['race'] = enc.fit_transform(df[['race']]).toarray()
df['religion'] = enc.fit_transform(df[['religion']]).toarray()
df.reset_index(inplace=True,drop=True)

In [16]:
X = df.drop(columns=['voted']).values
y = df['voted'].values

In [17]:
df.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,30,2,1,4,1.0,1,3,0.0,0.0,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,20,4,2,3,1.0,1,1,1.0,0.0,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30,3,1,3,0.0,1,2,0.0,0.0,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20,4,2,0,1.0,1,1,1.0,0.0,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,20,3,1,2,0.0,1,2,0.0,1.0,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1


이번 세션에서 배운 Cross validation과 Hyperparameter Optimization 모델을 활용하여  
0.69 이상의 validation score를 기록해보세요!

In [74]:
def lgbm_cv(colsample_bytree, 
            learning_rate,
            max_depth,
            n_estimators,
            num_leaves,
          x_data=None, y_data=None, n_splits=5, output='score'):
    
    score = 0
    kf = KFold(n_splits=n_splits, random_state=4321)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data[train_index], y_data[train_index]
        x_valid, y_valid = x_data[valid_index], y_data[valid_index]

        model = LGBMClassifier(
            colsample_bytree = colsample_bytree,
            learning_rate = learning_rate,
            max_depth = int(max_depth),
            n_estimators = int(n_estimators),
            num_leaves = int(num_leaves)                        
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        score += accuracy_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [89]:
func_fixed = partial(lgbm_cv, x_data=X, y_data=y, n_splits=5, output='score')
regBO = BayesianOptimization(
    func_fixed, 
    {
         'colsample_bytree' : (0.4,1),
         'learning_rate':(0.0001, 0.1),
         "max_depth" : (0,500),
         "n_estimators" : (15,25),
         "num_leaves" : (5,400)

    },
    random_state=1234
)

regBO.maximize(init_points=100, n_iter=0)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------------------
|  1        |  0.6804   |  0.5149   |  0.06225  |  218.9    |  22.85    |  313.1    |
|  2        |  0.6726   |  0.5636   |  0.02772  |  400.9    |  24.58    |  351.0    |
|  3        |  0.6837   |  0.6147   |  0.05015  |  341.7    |  22.13    |  151.2    |
|  4        |  0.6862   |  0.7367   |  0.05036  |  6.884    |  22.73    |  353.6    |
|  5        |  0.6788   |  0.6189   |  0.06158  |  37.69    |  18.69    |  373.6    |
|  6        |  0.6806   |  0.7908   |  0.03978  |  394.4    |  18.17    |  229.4    |
|  7        |  0.6768   |  0.9215   |  0.04367  |  401.1    |  16.44    |  283.2    |
|  8        |  0.6621   |  0.8227   |  0.02196  |  462.4    |  19.42    |  364.2    |
|  9        |  0.6368   |  0.4359   |  0.01851  |  23.68    |  21.75    |  239.9    |
|  10       |  0.5468   |  0.72     |  0.004428 |  280

In [90]:
params = regBO.max['params']

In [91]:
params

{'colsample_bytree': 0.9630733577179893,
 'learning_rate': 0.09387038130578335,
 'max_depth': 372.5222735539388,
 'n_estimators': 24.107350428473104,
 'num_leaves': 98.70375997384143}

In [94]:
models = lgbm_cv(
    params['colsample_bytree'], 
    params['learning_rate'],
    params["max_depth"],
    params["n_estimators"],
    params["num_leaves"],
    x_data=X, y_data=y, n_splits=5, output='model')